In [147]:
import numpy as np
import pandas as pd
import torch_geometric
import networkx as nx
from torch_geometric.data import Data, Dataset
from torch_geometric.datasets.planetoid import Planetoid
from torch_geometric.transforms.to_undirected import ToUndirected
import torch
import os

In [148]:
circles = {}
for file in os.listdir('/home/jrm28/fairness/subgraph_sketching-original/dataset/ego-gplus/raw/gplus/'):
    file_split = file.split('.') 
    circle = file_split[0]
    file_type = file_split[1]
    if circle not in circles:
        circles[circle] = dict()
    circles[circle][file_type] = '/home/jrm28/fairness/subgraph_sketching-original/dataset/ego-gplus/raw/gplus/' + file
    
for circle in circles.keys():
    # print(circles[circle])
    edge_list = pd.read_csv(circles[circle]['edges'], delimiter = ' ', header = None)
    
    # unique_values = edge_list.stack().unique()
    # unique_values_sorted = np.sort(unique_values)
    # node_mapper = dict(zip(unique_values_sorted, range(len(unique_values_sorted))))
    # edge_list[0] = edge_list[1].map(node_mapper)
    # edge_list[1] = edge_list[1].map(node_mapper)
    # edge_array = edge_list.sort_values(by=[0, 1]).values
    features = np.loadtxt(circles[circle]['feat'])
    
    if np.unique(edge_list.values.flatten()).shape[0] == 4417:
        print(np.unique(edge_list.values.flatten()).shape, len(features))
        print(circle)
    # sort features by node id
    # sorted_idx = features[:, 0].argsort()
    # features = features[sorted_idx, :]
    

KeyboardInterrupt: 

In [ ]:
circles['111091089527727420853']

{'featnames': '/home/jrm28/fairness/subgraph_sketching-original/dataset/ego-gplus/raw/gplus/111091089527727420853.featnames',
 'circles': '/home/jrm28/fairness/subgraph_sketching-original/dataset/ego-gplus/raw/gplus/111091089527727420853.circles',
 'egofeat': '/home/jrm28/fairness/subgraph_sketching-original/dataset/ego-gplus/raw/gplus/111091089527727420853.egofeat',
 'feat': '/home/jrm28/fairness/subgraph_sketching-original/dataset/ego-gplus/raw/gplus/111091089527727420853.feat',
 'edges': '/home/jrm28/fairness/subgraph_sketching-original/dataset/ego-gplus/raw/gplus/111091089527727420853.edges',
 'followers': '/home/jrm28/fairness/subgraph_sketching-original/dataset/ego-gplus/raw/gplus/111091089527727420853.followers'}

In [201]:
features = pd.read_csv(circles['111091089527727420853']['feat'], delimiter = ' ', header = None)
features = features[features[3] != 1] # Removing nodes which sensitive attribute is not available
features

,0,1,2,3,4,5,6,7,8,9,...,3520,3521,3522,3523,3524,3525,3526,3527,3528,3529
0,100859955896185685120,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,100492431527288250143,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,112399767740508618350,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,107332622748446972466,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,104843347600101507815,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959,117300752739997658443,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4960,116561012331692075800,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4961,110467702883125654398,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4962,109859000123225450719,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [159]:
node_mapper = dict(zip(features[0].values, range(features.shape[0])))

In [160]:
node_sens_attr = dict(zip(range(features.shape[0]), features[1]))

In [202]:
node_sens_attr = dict(zip(features[0], features[2]))

In [200]:
aux = list(map(mapping_fn, node_sens_attr))


[214,
 127,
 3296,
 1944,
 1308,
 4791,
 4410,
 2890,
 97,
 2963,
 2728,
 2246,
 3851,
 897,
 4181,
 954,
 941,
 453,
 4617,
 96,
 4021,
 2223,
 2549,
 2803,
 2841,
 884,
 1315,
 101,
 4035,
 48,
 1528,
 997,
 4204,
 3676,
 3821,
 4004,
 1900,
 4603,
 2569,
 4832,
 3727,
 715,
 184,
 2000,
 1291,
 2703,
 4597,
 1668,
 1514,
 4067,
 1208,
 2047,
 3018,
 3751,
 1021,
 4255,
 4108,
 4152,
 677,
 4717,
 3061,
 2149,
 446,
 1773,
 613,
 2490,
 3579,
 145,
 756,
 3610,
 497,
 4199,
 2353,
 2310,
 963,
 3744,
 2899,
 3572,
 3362,
 3915,
 1485,
 3899,
 19,
 359,
 3775,
 2456,
 722,
 509,
 4550,
 4124,
 1786,
 1703,
 3857,
 2104,
 3840,
 4344,
 459,
 4584,
 2038,
 4757,
 3801,
 4586,
 1882,
 523,
 2877,
 3908,
 2599,
 675,
 1586,
 758,
 1298,
 790,
 3308,
 4070,
 945,
 1877,
 2051,
 4669,
 138,
 3154,
 1328,
 2014,
 1534,
 2616,
 803,
 730,
 2852,
 3180,
 2152,
 796,
 1837,
 686,
 3557,
 4658,
 1934,
 1727,
 4434,
 3444,
 1948,
 4115,
 927,
 923,
 4750,
 2100,
 4672,
 764,
 1111,
 2662,
 1663,


In [214]:
import numpy as np

edge_list = pd.read_csv(circles['111091089527727420853']['edges'], delimiter = ' ', header = None)

all_nodes = np.unique(edge_list.values.flatten())
node_mapper = dict(zip(all_nodes, range(len(all_nodes))))

mapping_fn = lambda x: node_mapper[x] if x in node_mapper else None

edge_list[0] = edge_list[0].map(mapping_fn)
edge_list[1] = edge_list[1].map(mapping_fn)

edge_list = edge_list.dropna()

features = pd.read_csv(circles['111091089527727420853']['feat'], delimiter = ' ', header = None)
# features = features[features[3] != 1] # Removing nodes which sensitive attribute is not available

features[0] = features[0].map(mapping_fn)
features.sort_values(by=[0], inplace=True)
features = features.dropna()

sens_attrs = list(features[2])

KeyError: 1175

In [239]:
transform = ToUndirected()

# feature 1 will be the sensitive attribute
x = torch.ones(len(sens_attrs), dtype=torch.float32)
y = torch.tensor(sens_attrs)

gplus = Data(x=x, y=y, edge_index=torch.tensor(edge_list.values.T))
gplus = transform(gplus)

torch.save(gplus, '/home/jrm28/fairness/NeuralCommonNeighbor/dataset/gplus/processed/gplus_111091089527727420853.pt')

## EDA

In [232]:
y[gplus.edge_index].sum(0)

tensor([1, 0, 0,  ..., 0, 0, 0])

In [173]:
torch.vstack((y[gplus.edge_index])).sum(axis=0)  

TypeError: only integer tensors of a single element can be converted to an index

In [234]:
(y[gplus.edge_index].sum(0) == 0).sum() + (y[gplus.edge_index].sum(0) == 2).sum(), (y[gplus.edge_index].sum(0) == 1).sum(), 

(tensor(570860), tensor(524986))

In [237]:
(y[gplus.edge_index].sum(0) == 2).sum() / gplus.edge_index.shape[1]

tensor(0.3646)